In [10]:
from ipywidgets import interact

In [1]:
from utils import *
import numpy as np
from matplotlib import pyplot as plt

from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_psnr as psnr
from skimage.measure import compare_mse as mse
from skimage.measure import compare_nrmse as nrmse

Using TensorFlow backend.


In [2]:
train_low, train_high, val_low, val_high, test_low = data_loader_v2('../MRI Enhancing/4gan_m')
print("Train X's shape : ", train_low.shape)
print("Train Y's shape : ", train_high.shape)
print("Validation X's shape : ", val_low.shape)
print("Validation Y's shape : ", val_high.shape)
print("Test X's shape : ", test_low.shape)

Train X's shape :  (303, 320, 256, 3)
Train Y's shape :  (303, 320, 256, 6)
Validation X's shape :  (25, 320, 256, 3)
Validation Y's shape :  (25, 320, 256, 6)
Test X's shape :  (328, 320, 256, 3)


In [3]:
test = np.zeros((25, 320, 256, 12))

test[1:,...,:3] = val_high[:-1,...,3:]
test[...,3:-3] = val_high.copy()
test[:-1,...,-3:] = val_high[1:,...,:3]

In [17]:
def plot_fn_single(img):
    mdx = len(img)-1
    def plot(idx=0):
        plt.figure(figsize=(20, 10))
        for i in range(12):
            plt.subplot(2, 6, i+1)
            plt.imshow(img[idx, ..., i], cmap='gray', vmin=0, vmax=img[idx].max())
            plt.xlabel("%f ~ %f"%(0, img[idx, ..., i].max()))
    interact(plot, idx=(0, mdx, 1))

In [18]:
plot_fn_single(test)

interactive(children=(IntSlider(value=0, description='idx', max=24), Output()), _dom_classes=('widget-interact…

In [ ]:
ssim(train_high[0,...,0], train_low[0,...,0])

$$
SSIM(x, y) = \frac{(2\mu_x\mu_y+C_1)(2\sigma_{xy}+C_2)}{(\mu^2_x+\mu^2_y+C_1)(\sigma^2_x+\sigma^2_y+C_2)}
$$

In [ ]:
SSIM(train_high[0:1,...,0:1], train_low[0:1,...,0:1], data_range=65535)

In [ ]:
def SSIM(y_true, y_pred, data_range=65535):
    _, h, w, _ = y_true.shape
    C1 = (0.01*data_range)**2
    C2 = (0.03*data_range)**2
    
    mu_true = np.mean(y_true, axis=(1,2), keepdims=True)
    mu_pred = np.mean(y_pred, axis=(1,2), keepdims=True)
    
    std_true = np.std(y_true, axis=(1,2))
    std_pred = np.std(y_pred, axis=(1,2))
    
    cov = np.sum((y_true - mu_true)*(y_pred - mu_pred), axis=(1, 2))/(h*w-1)
    
    A = 2*mu_true.squeeze(axis=(1, 2))*mu_pred.squeeze(axis=(1, 2)) + C1
    B = 2*cov + C2
    C = mu_true.squeeze(axis=(1, 2))**2 + mu_pred.squeeze(axis=(1, 2))**2 + C1
    D = std_true**2 + std_pred**2 + C2
    
    return (A+B+1)/(C+D+1)